## Greedy Algorithms

### Money Change

Given an integer $1 \le money \le 10^3$, find
the minimum number of coins with
denominations 1, 5, and 10 that changes $money$.

In this problem, you will implement a simple greedy algorithm used by cashiers all over the world. We assume that a cashier has unlimited number of coins of each denomination.


In [1]:
def money_change(money):
    assert 0 <= money <= 10 ** 3
    return int(money / 10) + int((money % 10) / 5) + int(money % 5)

if __name__ == '__main__':
    input_money = int(input())
    print(money_change(input_money))

159
20


Alternative solution that I have found earlier

In [6]:
def money_change(money):
    assert 0 <= money <= 10 ** 3

    count = 0
    if money >= 10:
        count += int(money/10)
    if money%10 >= 5:
        count += 1
    count += money%5
    return count

if __name__ == '__main__':
    input_money = int(input())
    print(money_change(input_money))

36
5


### Car Fueling

Compute the minimum number of tank refills to get from one city to another.

Assuming that the distance between the cities is $1 \le d \le 10^5$ miles,
a car can travel at most $1 \le m \le 400$ miles on a full tank,
and there are $1 \le n \le 300$ gas stations at distances
$stop_1,stop_2,\dotsc,stop_n$ along the way,
output the minimum number of refills needed.
Assume that the car starts with a full tank.
If it is not possible to reach the destination, output −1.
The distances to gas stations satisfy the inequalities
$$0 < stop_1 < stop_2 < \dotsb < stop_n < d .$$


In [3]:
def compute_min_number_of_refills(d, m, stops):
    assert 1 <= d <= 10 ** 5
    assert 1 <= m <= 400
    assert 1 <= len(stops) <= 300
    assert 0 < stops[0] and all(stops[i] < stops[i + 1] for i in range(len(stops) - 1)) and stops[-1] < d
    position = 0
    refill = []
    i = 0
    while i < len(stops):
        if stops[0] > m or d - stops[-1] > m or (i > 0 and stops[i] - stops[i - 1] > m):
            return -1
        if stops[i] - position < m:
            i += 1
            continue
        if stops[i] - position == m:
            position = stops[i]
            refill.append(position)
            i += 1
            continue
        if stops[i] - position > m:
            position = stops[i-1]
            refill.append(position)
    if d - position <= m:
        return len(refill)
    else:
        refill.append(stops[-1])
        return len(refill)

if __name__ == '__main__':
    input_d = int(input())
    input_m = int(input())
    input_n = int(input())
    input_stops = list(map(int, input().split()))
    assert len(input_stops) == input_n
    print(compute_min_number_of_refills(input_d, input_m, input_stops))

1000
200
10
50 70 150 250 300 340 510 590 700 810
5


Alternative solution that I have found earlier

In [4]:
def compute_min_number_of_refills(d, m, stops):
    assert 1 <= d <= 10 ** 5
    assert 1 <= m <= 400
    assert 1 <= len(stops) <= 300
    assert 0 < stops[0] and all(stops[i] < stops[i + 1] for i in range(len(stops) - 1)) and stops[-1] < d

    points = []
    station = 0
    amount = m
    
    while station <= len(stops)-1:
        if stops[0] > m or d - stops[-1] > m or (station > 0 and stops[station] - stops[station-1] > m):
            return "IMPOSSIBLE"
        if station == 0:
            amount = amount - stops[station]
            if amount == 0:
                points.append(stops[station])
                if points[-1] >= d - m:
                    break
                amount = m
        if station > 0 and station <= len(stops)-1:
            amount = amount - (stops[station] - stops[station-1])
            if amount == 0:
                points.append(stops[station])
                if points[-1] >= d - m:
                    break
                amount = m
            if amount < 0:
                points.append(stops[station-1])
                if points[-1] >= d - m:
                    break
                amount = m - (stops[station] - stops[station-1])
        if station == len(stops)-1 and d - stops[station] > amount:
            points.append(stops[station])
        station += 1
    return len(points),points

if __name__ == '__main__':
    input_d = int(input())
    input_m = int(input())
    
    input_n = int(input())
    input_stops = list(map(int, input().split()))
    assert len(input_stops) == input_n
    print(compute_min_number_of_refills(input_d, input_m, input_stops))

12
3
8
1 2 4 5 8 9 10 11
(4, [2, 5, 8, 11])


### Maximum Value of the Loot (Fractional Knapsack Problem)


Given the capacity of a backpack
as well as the weights and per pound prices
of $n$ different compounds, compute the
maximum total price of items that
fit into the backpack of the given capacity.

A thief breaks into a spice shop and finds 
four pounds of saffron, three pounds of 
vanilla, and five pounds of cinnamon. His backpack 
fits at most nine pounds, therefore he cannot 
take everything. Assuming that the prices of 
saffron, vanilla, and cinnamon are 
5000, 200, and 10 per pound respectively, 
what is the most valuable loot in this case? 
If the thief takes $u_1$ pounds of saffron, 
$u_2$ pounds of vanilla, and $u_3$ pounds of cinnamon, 
the total price of the loot is 
$5\,000 \cdot u_1 + 200 \cdot u_2 + 10\cdot u_3$. 
The thief would like to maximize the value of this expression 
subject to the following constraints: 
$u_1 \le 4$, $u_2 \le 3$, $u_3 \le 5$, $u_1+u_2+u_3 \le 9$.


In [6]:
def maximum_loot_value(capacity, weights, prices):
    assert 0 <= capacity <= 2 * 10 ** 6
    assert len(weights) == len(prices)
    assert 1 <= len(weights) <= 10 ** 3
    assert all(0 < w <= 2 * 10 ** 6 for w in weights)
    assert all(0 <= p <= 2 * 10 ** 6 for p in prices)
    unit_price = []
    for i in range(len(prices)):
        unit_price.append(prices[i]/weights[i])
    combined = []
    for i in range(len(prices)):
        combined.append([weights[i],unit_price[i]])
    sorted_combined = sorted(combined, key = lambda x: x[1],  reverse = True)
    amount = 0
    for i in range(len(sorted_combined)):
        if capacity - sorted_combined[i][0] >= 0:
            capacity -= sorted_combined[i][0]
            amount += sorted_combined[i][0] * sorted_combined[i][1]
            continue
        amount += capacity * sorted_combined[i][1]
        break
    return amount

if __name__ == "__main__":
    n, input_capacity = map(int, input().split())
    input_prices = []
    input_weights = []
    for i in range(n):
        inputs = tuple(map(int, input().split()))
        input_prices.append(inputs[0])
        input_weights.append(inputs[1])
    opt_value = maximum_loot_value(input_capacity, input_weights, input_prices)
    print("{:.10f}".format(opt_value))

3 100
120 60
200 50
120 40
340.0000000000


Alternative solution that I have found earlier

In [4]:
def maximum_loot_value(capacity, weights, prices):
    assert 0 <= capacity <= 2 * 10 ** 6
    assert len(weights) == len(prices)
    assert 1 <= len(weights) <= 10 ** 3
    assert all(0 < w <= 2 * 10 ** 6 for w in weights)
    assert all(0 <= p <= 2 * 10 ** 6 for p in prices)

    whole = []
    for i in range(len(prices)):
        whole.append([prices[i]/weights[i],weights[i],prices[i]])
    final = sorted(whole,reverse=True)
    total_price = 0
    index = 0
    while capacity > 0:
        if capacity >= final[index][1]:
            total_price += final[index][2]
        else:
            total_price += capacity * final[index][0]
            break
        capacity -= final[index][1]
        index += 1
    return total_price

if __name__ == "__main__":
    n = int(input())
    input_capacity = int(input())
    input_prices = list(map(int, input().split()))
    input_weights = list(map(int, input().split()))
    assert len(input_prices) == len(input_weights) == n
    opt_value = maximum_loot_value(input_capacity, input_weights, input_prices)
    print("{:.2f}".format(opt_value))

3
30
180 100 76
68 15 13
181.29


### Maximum Advertisement Revenue

You have $n=3$ advertisement slots on your popular Internet page and 
you want to sell them to advertisers. They expect, respectively, 
${clicks}_1=10$, 
${clicks}_2=20$, and ${clicks}_3=30$ clicks per day. You found three advertisers 
willing to pay ${price}_1=2$, ${price}_2=3$, and ${price}_3=5$ per click. 
How would you pair the slots and advertisers? For example, the blue pairing 
gives a revenue of $10 \cdot 5 + 20 \cdot 2 + 30 \cdot 3 = 180$ dollars, while 
the black one results in revenue of $10\cdot 3 + 20 \cdot 5 + 30 \cdot 2=190$ dollars.


Find the maximum dot product of two
sequences of numbers.
* Input: Two sequences of $n$ positive
integers: $price_1,\dots,price_n$
and $clicks_1,\dots,clicks_n$.
* Output: The maximum value of
$price_1 \cdot c_1 + \dots+price_n \cdot c_n$,
where $c_1,\dots,c_n$ is a
permutation of $clicks_1,\dots,clicks_n$.


In [81]:
from itertools import permutations

def max_dot_product_naive(first_sequence, second_sequence):
    assert len(first_sequence) == len(second_sequence)
    assert len(first_sequence) <= 10 ** 3
    assert all(0 <= f <= 10 ** 5 for f in first_sequence)
    assert all(0 <= s <= 10 ** 5 for s in second_sequence)
    
    max_product = 0
    for permutation in permutations(second_sequence):
        dot_product = sum(first_sequence[i] * permutation[i] for i in range(len(first_sequence)))
        max_product = max(max_product, dot_product)

    return max_product


def max_dot_product(first_sequence, second_sequence):
    assert len(first_sequence) == len(second_sequence)
    assert len(first_sequence) <= 10 ** 3
    assert all(0 <= f <= 10 ** 5 for f in first_sequence)
    assert all(0 <= s <= 10 ** 5 for s in second_sequence)
    
    return sum([i * j for i,j in zip(sorted(first_sequence), sorted(second_sequence))])

if __name__ == '__main__':
    n = int(input())
    prices = list(map(int, input().split()))
    clicks = list(map(int, input().split()))
    assert len(prices) == len(clicks) == n
    print(max_dot_product(prices, clicks))

5
12 543 12 6 132
234 765 21 76 0
447447


Alternative solution that I have found earlier

In [7]:
def max_dot_product(first_sequence, second_sequence):
    assert len(first_sequence) == len(second_sequence)
    assert len(first_sequence) <= 10 ** 3
    assert all(0 <= f <= 10 ** 5 for f in first_sequence)
    assert all(0 <= s <= 10 ** 5 for s in second_sequence)

    price = sorted(first_sequence)
    click = sorted(second_sequence)
    max_dot_product = 0
    for i in range(len(price)):
        max_dot_product += price[i]*click[i]
    return max_dot_product

if __name__ == '__main__':
    n = int(input())
    prices = list(map(int, input().split()))
    clicks = list(map(int, input().split()))
    assert len(prices) == len(clicks) == n
    print(max_dot_product(prices, clicks))

3
10 20 30
2 3 5
230


### Collecting Signatures

You are responsible for collecting signatures from all tenants in a building.
For each tenant, you know a period of time when he or she is at home.
You would like to collect all signatures by visiting the building as
few times as possible. For simplicity, we assume that when you enter
the building, you instantly collect the signatures of all tenants that
are in the building at that time.

**Input**: A sequence of $n \le 10^3$ segments
$[l_1,r_1],...,[l_n,r_n]$ on a line.

**Output**: A set of points of minimum size
such that each segment $[l_i,r_i]$ contains a point,
i.e., there exists a point $x$ such that $l_i \le x \le r_i$.

In [2]:
def compute_optimal_points(segments):
    position = []
    segments = sorted(segments)
    for i in range(len(segments)):
        if len(position) == 0:
            position.append(segments[i][1])
        if segments[i][1] < position[-1]:
            position = position[:-1]
        if len(position) != 0 and position[-1] >= segments[i][0]:
            continue
        position.append(segments[i][1])
    return position

if __name__ == '__main__':
    n = int(input())
    assert n >= 2
    input_segments = []
    for i in range(n):
        inputs = tuple(map(int, input().split()))
        input_segments.append(inputs)
    output_points = compute_optimal_points(input_segments)
    print(len(output_points))
    print(*output_points)

4
5 10
5 9
6 7
8 9
2
7 9


Alternative solution that I have found earlier

In [5]:
#from collections import namedtuple
#Segment = namedtuple('Segment', 'start end')

def compute_optimal_points(segments):
    
    segments.sort(key = lambda x:x[0])
    start = [x[0] for x in segments]
    end = [x[1] for x in segments]
    
    output = []
    condition = True
    temp = end[0]
    
    for i in range(0,len(segments)):
        if i != len(segments)-1 and start[i] == start[i+1]:
            temp = min(temp,end[i+1])
            continue
        if condition:
            output.append(temp)
            condition = False
        if i != len(segments)-1 and output[-1] >= start[i+1]:
            continue
        if i != len(segments)-1 and output[-1] < start[i+1]:
            temp = end[i+1]
            condition = True
        if i == len(segments)-1:
            if output[-1] < start[i]:
                output.append(temp)                
    return output

def alternative_main():
    n, *data = map(int, input().split())
    segments = list(map(lambda x: Segment(x[0], x[1]), zip(data[::2], data[1::2])))
    points = compute_optimal_points(segments)
    print(len(points))
    print(*points)
    
if __name__ == '__main__':
    n = int(input())
    assert n >= 2
    input_segments = []
    for i in range(n):
        inputs = tuple(map(int, input().split()))
        assert len(inputs) == 2
        assert inputs[0] < inputs[1]
        input_segments.append(inputs)
    output_points = compute_optimal_points(input_segments)
    print(len(output_points))
    print(output_points)

12
1 17
4 9
1 5
20 22
4 6
12 14
8 12
8 20
3 9
7 10
16 19
7 16
5
[5, 10, 14, 19, 22]


### Maximum Number of Prizes

You are organizing a competition for 
children and have $n$ candies to give as prizes. 
You would like to use these candies for top $k$ places 
in a competition with a restriction that a higher place gets 
a larger number of candies. To make as many children happy 
as possible, you need to find the largest value of $k$ for 
which it is possible.

**Input**: An integer $1 \le n \le 10^9$.

**Output**: The maximum 
number $k$ such that $n$ can be represented as the sum of $k$ pairwise 
distinct positive integers and these integers (if there are many such 
representations, output any of them).


In [1]:
def compute_optimal_summands(n):
    assert 1 <= n <= 10 ** 9
    summands = []
    i = 1
    while n > 0:
        if len(summands) == 0:
            summands.append(i)
        n -= i
        if n > summands[-1]:
            i += 1
            summands.append(i)
        else:
            summands[-1] += n
            break
    return summands


if __name__ == '__main__':
    input_n = int(input())
    output_summands = compute_optimal_summands(input_n)
    print(len(output_summands))
    print(*output_summands)

100
13
1 2 3 4 5 6 7 8 9 10 11 12 22


Alternative solution that I found earlier

In [2]:
def compute_optimal_summands(n):
    assert 1 <= n <= 10 ** 9
    
    summands = []
    if n == 1 or n == 2:
        summands = [n]
        return summands
    while n > 0:
        if len(summands) == 0:
            summands.append(1)
            n -= 1
        if n-(summands[-1]+1) >= summands[-1]+2:
            summands.append(summands[-1]+1)
            n -= summands[-1]
            continue
        summands.append(n)
        break
    return summands

if __name__ == '__main__':
    input_n = int(input())
    output_summands = compute_optimal_summands(input_n)
    print(len(output_summands))
    print(output_summands)

76
11
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 21]


### Maximum Salary

This is probably the most important problem 
in this course :). 
As the last question of an interview, 
your future boss gives you a few pieces of 
paper with a single number written on 
each of them and asks you to compose 
a largest number from these numbers. 
The resulting number is going to be 
your salary, so you are very motivated to 
solve this problem!

**Input**: Integers 
$1 \le a_1, a_2, \dotsc, a_n \le 10^3$, where
$1 \le n \le 100$.

**Output**: The largest number that 
can be composed out of $a_1, \dotsc, a_n$.


In [3]:
from itertools import permutations

def largest_number_naive(numbers):
    numbers = list(map(str, numbers))
    largest = 0
    for permutation in permutations(numbers):
        largest = max(largest, int("".join(permutation)))
    return largest

def largest_number(numbers):
    assert all(1 <= int(a) <= 10 ** 4 for a in numbers)
    assert 1 <= len(numbers) <= 100
    numbers = list(map(str, numbers))
    i = 1
    while i <= len(numbers) - 1:
        j = i
        while j >= 1:
            if int(numbers[j-1] + numbers[j]) < int(numbers[j] + numbers[j-1]):
                numbers[j], numbers[j-1] = numbers[j-1], numbers[j]
                j -= 1
            else:
                break
        i += 1
    return int(''.join(map(str, numbers)))

if __name__ == '__main__':
    n = int(input())
    input_numbers = input().split()
    assert len(input_numbers) == n
    print(largest_number(input_numbers))

8
2 21 23 211 213 231 232 234
23423232231221321211


In [4]:
import random
import time

def StressTest(N, M):
    assert 0 <= N <= 100
    assert 1 <= M <= 10000
    while True:
        n = random.randint(1, N)
        A = [random.randint(1, M) for i in range(0, n)]
        print(A)
        result1 = largest_number_naive(A)
        result2 = largest_number(A)
        if result1 == result2:
            print('OK', 'result1, result2 = ', result1)
        else:
            print('Answer is wrong:', 'result1 = ', result1, 'result2 = ', result2)
            break

def ComplexityTest(N, M):
    assert 1 <= N <= 100
    assert 1 <= M <= 10000
    start_time = time.time()
    n = random.randint(1, N)
    A = [random.randint(1, M) for i in range(0, n)]
    print(A)
    print(largest_number(A))
    print("--- %s seconds ---" % (time.time() - start_time))

    StressTest() passed for thousands of cases. So, algorithm works properly. Now, let's check the running time.

In [6]:
ComplexityTest(100,10000)

[139, 7989, 1003, 2241, 7522, 4768, 9470, 212, 1977, 4107, 9680, 6328, 4583, 614, 6487, 8371, 543, 759, 9372, 9306, 5545, 8031, 2671, 4555, 3388, 6658, 6198, 1307, 8101, 7000, 1618, 3264, 6485, 3567, 7301, 1207, 8936, 8186, 6728, 5919, 5794, 3775, 9329, 4812, 4653, 4024, 1513, 2913, 6893, 3466, 2677, 3076, 468, 7680, 1237, 715, 3739, 6633, 2461, 373, 3136, 3997, 9063, 2273, 4816, 6144, 4098, 929, 7822, 5861, 9432, 5028, 7146, 5858, 7692, 1905, 6128, 5443, 8765, 4244, 6773]
968094709432937293299306929906389368765837181868101803179897822769276807597522730171571467000689367736728665866336487648563286198614614461285919586158585794554554435435028481648124768468465345834555424441074098402439973775373937335673466338832643136307629132677267124612273224121219771905161815131391307123712071003
--- 0.001997709274291992 seconds ---


Alternative solution that I found earlier

In [7]:
def largest_number(numbers):
    assert all(1 <= a <= 10 ** 4 for a in numbers)
    assert 1 <= len(numbers) <= 100
    
    num_sort = sorted(numbers,key=lambda x: x/(10**(len(str(x))-1)),reverse=True)
    index = 0
    result = int(''.join(map(str,num_sort)))
    
    while index < len(numbers):
        temp = index
        a = list(num_sort)
        while temp != 0:
            flip = a[temp-1]
            a[temp-1] = a[temp]
            a[temp] = flip
            if result > int(''.join(map(str,a))):
                temp -= 1
                continue
            if result < int(''.join(map(str,a))):
                result = int(''.join(map(str,a)))
                num_sort = list(a)
                temp -= 1
        index += 1
    return result
    
if __name__ == '__main__':
    n = int(input())
    input_numbers = list(map(int,input().split()))
    assert len(input_numbers) == n
    print(largest_number(input_numbers))

17
6 62 68 683 689 6837 6611 2 21 23 211 213 231 232 233 5 1
68968683768366611625233232322312213212111
